# Importing Libraries

In [1]:
import socket
import threading

import sounddevice as sd
import numpy as np

# Utility Functions

In [2]:
def recordVoice(fs ,duration): # fs : frequency Sampling
    print("<-- Recording Voice -->")
    myrecording = sd.rec(int(duration * fs), channels=1)
    sd.wait()
    
    toString = myrecording.tostring() # converting recording to string
    
    return toString

def sendVoice(socket, stringVoice):
    print("<-- Sent Voice -->")
    socket.send(stringVoice)
    
def receiveVoice(socket, fs ,duration):
    print("<-- Received Voice -->")
    msg = s.recv(int(fs) * int(duration) * 4)
    
    return msg

def playVoice(stringVoice):
    voice = np.frombuffer(stringVoice,dtype=np.float32)
    print("<-- Playing Voice -->")
    sd.play(voice)

# Multi threaded Classes

In [3]:
class send(threading.Thread): 
    def __init__(self, socket, fs, duration): # fs : frequency of sampling
        threading.Thread.__init__(self) 
        self.socket = socket
        self.fs = fs
        self.duration = duration
        
        self._running = True
        
    def terminate(self):
        self._running = False

    def run(self): 
        while self._running:
            name = input("> Enter Any Key To Record And Send Voice <")
            
            voice = recordVoice(self.fs , self.duration)
            sendVoice(self.socket, voice)


class receive(threading.Thread): 
    def __init__(self, socket, fs, duration): 
        threading.Thread.__init__(self) 
        self.socket = socket
        self.fs = fs
        self.duration = duration
        
        self._running = True
        
    def terminate(self):
        self._running = False

    def run(self): 
        while self._running:
            recievedVoice = self.socket.recv(int(fs) * int(duration) * 4)
            print("<-- Received Voice -->")
            playVoice(recievedVoice)


# Defining Some Value

In [4]:
fs = 44200 # Sampling rate (Sampling Rate Must be same across Client And Server)
sd.default.samplerate = fs # Setting Default Sampling Rate

duration = 5 # sec

# Socket Code

In [ ]:
#Listening for socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind((socket.gethostname(), 20001))

threads1 = []
threads2 = []


print("...Waiting for clients...")

while True:
    s.listen(5)
    clientsocket, address = s.accept()
    print("New connection : " + str(clientsocket))
    
    thread1 = send(clientsocket, fs, duration)
    thread1.start() 
    threads1.append(thread1)
    
    thread2 = receive(clientsocket, fs, duration)
    thread2.start() 
    threads2.append(thread2)
    

for thread in threads1:
    thread.join()

for thread in threads2:
    thread.join()


...Waiting for clients...
New connection : <socket.socket fd=1524, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.0.65', 20001), raddr=('192.168.0.65', 61567)>
> Enter Any Key To Record And Send Voice <ha
<-- Recording Voice -->
<-- Sent Voice -->
